In [ ]:
!pip install transformers

!pip install datasets

!pip install scikit-learn

!pip install torch

!pip install torch --extra-index-url https://download.pytorch.org/whl/cu118


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim import AdamW

os.environ['WANDB_DISABLED'] = 'true'

tokenizer = AutoTokenizer.from_pretrained("ltrctelugu/ltrc-roberta")
model = AutoModelForSequenceClassification.from_pretrained("ltrctelugu/ltrc-roberta", num_labels=3)

dataset = load_dataset("mounikaiiith/Telugu_Sentiment")

def encode_labels(example):
    label_mapping = {'neg': 0, 'neutral': 1, 'pos': 2}
    example['label'] = label_mapping[example['Sentiment']]
    return example

encoded_dataset = dataset.map(encode_labels)

def tokenize_function(example):
    return tokenizer(example['Sentence'], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = encoded_dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['validation']
test_dataset = tokenized_dataset['test']

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    adam_epsilon=1e-8

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), lr=3e-5, eps=1e-8), None)  # Specify AdamW optimizer
)

trainer.train()

train_results = trainer.evaluate(train_dataset)
print("Training set evaluation:", train_results)

val_results = trainer.evaluate(val_dataset)
print("Validation set evaluation:", val_results)

test_results = trainer.evaluate(test_dataset)
print("Test set evaluation:", test_results)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ltrctelugu/ltrc-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3510 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.013400,1.009058,0.534473,0.372325,0.285661,0.534473
2,1.003800,1.008998,0.534473,0.372325,0.285661,0.534473
3,1.006600,1.009527,0.534473,0.372325,0.285661,0.534473
4,1.017900,1.009705,0.534473,0.372325,0.285661,0.534473
5,0.995500,1.009398,0.534473,0.372325,0.285661,0.534473
6,1.009000,1.008363,0.534473,0.372325,0.285661,0.534473
7,1.005300,1.010725,0.534473,0.372325,0.285661,0.534473
8,1.006600,1.008906,0.534473,0.372325,0.285661,0.534473
9,0.983200,1.008974,0.534473,0.372325,0.285661,0.534473
10,1.013900,1.008537,0.534473,0.372325,0.285661,0.534473


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training set evaluation: {'eval_loss': 1.0050841569900513, 'eval_accuracy': 0.5371356559209723, 'eval_f1': 0.375392648983593, 'eval_precision': 0.2885147128616532, 'eval_recall': 0.5371356559209723, 'eval_runtime': 158.1819, 'eval_samples_per_second': 155.511, 'eval_steps_per_second': 4.861, 'epoch': 10.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation set evaluation: {'eval_loss': 1.0085371732711792, 'eval_accuracy': 0.5344729344729344, 'eval_f1': 0.37232500002644825, 'eval_precision': 0.2856613176841097, 'eval_recall': 0.5344729344729344, 'eval_runtime': 22.5643, 'eval_samples_per_second': 155.555, 'eval_steps_per_second': 4.875, 'epoch': 10.0}
Test set evaluation: {'eval_loss': 0.9981046915054321, 'eval_accuracy': 0.5430115171335135, 'eval_f1': 0.3821896395010851, 'eval_precision': 0.29486150773964, 'eval_recall': 0.5430115171335135, 'eval_runtime': 45.1929, 'eval_samples_per_second': 155.622, 'eval_steps_per_second': 4.868, 'epoch': 10.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
